#INIT

In [0]:
import pyspark.sql.functions as F
from pyspark.sql.types import StringType, DateType
from pyspark.sql.functions import col, trim, length

#Reading from bronze

In [0]:
df = spark.table("workspace.bronze.crm_sales_details")

#Silver transformations

In [0]:
#Trimming
for field in df.schema.fields:
    if isinstance(field.dataType, StringType):
        df = df.withColumn(field.name, trim(col(field.name)))

In [0]:
#Cleaning dates, transforms into date if not null and if the column length is equal to 8, otherwise returns NULL
df = (
    df
    .withColumn(
        "sls_order_dt",
        F.when(
            col("sls_order_dt").isNull() | (length(col("sls_order_dt")) != 8),
            None
        ).otherwise(F.to_date(col("sls_order_dt").cast("string"), "yyyyMMdd"))
    )
    .withColumn(
        "sls_ship_dt",
        F.when(
            col("sls_ship_dt").isNull() | (length(col("sls_ship_dt")) != 8),
            None
        ).otherwise(F.to_date(col("sls_ship_dt").cast("string"), "yyyyMMdd"))
    )
    .withColumn(
        "sls_due_dt",
        F.when(
            col("sls_due_dt").isNull() | (length(col("sls_due_dt")) != 8),
            None
        ).otherwise(F.to_date(col("sls_due_dt").cast("string"), "yyyyMMdd"))
    )
)


In [0]:
#Price correction using sales
df = (df.withColumn(
    "sls_price",
    F.when(col("sls_price").isNull() | (col("sls_price") <= 0),
           F.when(
               col("sls_quantity") !=0,
               col("sls_sales") / col("sls_quantity")
           ).otherwise("sls_price"))
    ))

In [0]:
#Renaming columns
RENAME_MAP = {
    "sls_ord_num": "order_number",
    "sls_prd_key": "product_number",
    "sls_cust_id": "customer_id",
    "sls_order_dt": "order_date",
    "sls_ship_dt": "ship_date",
    "sls_due_dt": "due_date",
    "sls_sales": "sales_amount",
    "sls_quantity": "quantity",
    "sls_price": "price"
}
for old_name, new_name in RENAME_MAP.items():
    df = df.withColumnRenamed(old_name, new_name)
     


In [0]:
df.write.mode("overwrite").format("delta").saveAsTable("workspace.silver.crm_sales_details")